In [ ]:
import numpy as np
from scipy.signal import butter, lfilter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling2D, MaxPooling1D, LSTM, Bidirectional, Flatten, Dense, Dropout, Input, GlobalAveragePooling1D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Resampling function to get all time series to the same number of steps for compatibility with neural networks
def resample(data, target_length=64):
    return np.interp(
        np.linspace(0, len(data), target_length),
        np.arange(len(data)),
        data)


def standardize_per_sample(data):
    means = data.mean(axis=1, keepdims=True)
    stds = data.std(axis=1, keepdims=True)
    # Add a small epsilon to avoid division by zero
    eps = 1e-8
    data_out = (data - means) / (stds + eps)

    return data_out


def normalise_data_per_sample(train, test):
    train_out = standardize_per_sample(train)
    test_out = standardize_per_sample(test)
    return train_out, test_out


# Preprocessing function for all samples(filtering)
def preprocess_data_filter_high(data):
    processed = []
    for sample in data:
        filtered_sample = np.array([butter_highpass_filter(feature) for feature in sample.T]).T
        processed.append(filtered_sample)
    processed = np.array(processed, dtype=np.float32)
    return processed


def preprocess_filter_moving_average(data, window_size=5):
    def moving_average_filter(signal, window_size):
        return np.convolve(signal, np.ones(window_size) / window_size, mode='same')

    processed = []
    for sample in data:
        filtered_sample = np.array([moving_average_filter(feature, window_size) for feature in sample.T]).T
        processed.append(filtered_sample)
    return np.array(processed, dtype=np.float32)


# Preprocessing function for all samples(resampling)
def preprocess_data_resample(data):
    processed = []
    for sample in data:
        resampled_sample = np.array([resample(feature) for feature in sample.T]).T
        processed.append(resampled_sample)
    processed = np.array(processed, dtype=np.float32) 
    return processed

# Function to plot confusion matrix. This will be used at the very end to visualise the errors in some predictions of the alphabets
def plot_confusion_matrix(y_true, y_pred, model_name, class_names):
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix for {model_name}")
    plt.xlabel("Predicted Alphabets")
    plt.ylabel("True Alphabets")
    plt.show()

# Load already splitted dataset
X = np.load('X_train_Shuffled.npy', allow_pickle=True)
y = np.load('y_train_Shuffled.npy', allow_pickle=True)

nac_X_train = np.load('nac_X_train.npy', allow_pickle=True)
nac_y_train = np.load('nac_y_train.npy', allow_pickle=True)
nac_X_test = np.load('nac_X_test.npy', allow_pickle=True)
nac_y_test = np.load('nac_y_test.npy', allow_pickle=True)

In [ ]:
#Checking the average length of the time series data in order to inform me on the number of neurons to use for each layer
#for the various NN models I have. Also to inform on resample size

sequence_lengths = [seq.shape[0] for seq in X_train]

# Calculate average number of time steps
average_time_steps = np.median(sequence_lengths)

print("average time steps: ",average_time_steps)

In [ ]:
# Choosing one sample to plot
sample_index = 0  
sample_data = X_train[sample_index]

#Below are the features in their respective order(acc=accelerometer, gyro=gyroscope each in 3 axis)
features = ['Accx','Accy','Accz','Gyrox','Gyroy','Gyroz']
X_train_heat = preprocess_data_resample(X_train)
print(X_train_heat.shape)

# Averaging features over time steps to get in order to visualise their correlations in the correlation matrix
data_averaged = X_train_heat.mean(axis=1)

# Computing the correlation matrix
correlation_matrix = np.corrcoef(data_averaged.T)

# Plotting the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", xticklabels=[f"{features[i]}" for i in range(6)], yticklabels=[f"{features[i]}" for i in range(6)])
plt.title("Correlation Matrix of Features (Averaged Across Time Steps)")
plt.show()

In [ ]:
# Load the two X_train and y_train datasets for nac
nac_X_train = preprocess_data_resample(nac_X_train)
nac_X_test = preprocess_data_resample(nac_X_test)

nac_X_train = preprocess_filter_moving_average(nac_X_train)
nac_X_test = preprocess_filter_moving_average(nac_X_test)

nac_X_train, nac_X_test = normalise_data_per_sample(nac_X_train,nac_X_test)

In [ ]:
#picking a random sample for visualisation
sample_index = 15
sample_data_raw = X_train[sample_index]

# Plotting each feature as a separate line
plt.figure(figsize=(12, 6))
for feature_idx in range(sample_data_raw.shape[1]):
    plt.plot(sample_data_raw[:, feature_idx], label=f"{features[feature_idx]}")
plt.title(f"Time-Series Data for Sample {sample_index} (Raw)")
plt.xlabel("Time Steps")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()


# resampling time steps to 52 based on ealier calculation
X_train = preprocess_data_resample(X_train) #resample
X_test = preprocess_data_resample(X_test)
sample_data_resample = X_train[sample_index]

# Showing nature of sensor values after resampling
plt.figure(figsize=(12, 6))
for feature_idx in range(sample_data_resample.shape[1]):
    plt.plot(sample_data_resample[:, feature_idx], label=f"{features[feature_idx]}")
plt.title(f"Time-Series Data for Sample {sample_index} (Resampled to 45 time steps)")
plt.xlabel("Time Steps")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()


# filtering data moving average
X_train = preprocess_filter_moving_average(X_train)
X_test = preprocess_filter_moving_average(X_test)


sample_data_filter = X_train[sample_index]

# Showing nature of data after filtering
plt.figure(figsize=(12, 6))
for feature_idx in range(sample_data_filter.shape[1]):
    plt.plot(sample_data_filter[:, feature_idx], label=f"{features[feature_idx]}")
plt.title(f"Time-Series Data for Sample {sample_index} (Filtered with moving average filter with window of 5)")
plt.xlabel("Time Steps")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()



# standardising features per sample
X_train, X_test = normalise_data_per_sample(X_train, X_test)
sample_data_normalised = X_train[sample_index]

# Showing nature of data after normalising
plt.figure(figsize=(12, 6))
for feature_idx in range(sample_data_normalised.shape[1]):
    plt.plot(sample_data_normalised[:, feature_idx], label=f"{features[feature_idx]}")
plt.title(f"Time-Series Data for Sample {sample_index} (normalised)")
plt.xlabel("Time Steps")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()


# # variance plot
import pandas as pd
def plot_imu_variance(
    x_train,
    x_test,
    sample_idx,
    source='train',
    window=5,
    threshold=0.5
):
    if source.lower() == 'train':
        sample = x_train[sample_idx]
        title_str = f"Mean Rolling Variance (window={window}) - Train Sample #{sample_idx}"
    else:
        sample = x_test[sample_idx]
        title_str = f"Mean Rolling Variance (window={window}) - Test Sample #{sample_idx}"

    columns = ['AccX', 'AccY', 'AccZ','GyroX','GyroY','GyroZ']
    df = pd.DataFrame(sample, columns=columns)

    rolling_var = df.rolling(window=window, center=True).var()

    variances = rolling_var.mean(axis=1)

    plt.figure(figsize=(12, 4))
    plt.plot(variances, label='Mean Rolling Variance')
    plt.title(title_str)
    plt.xlabel("Time Step")
    plt.ylabel("Mean Variance Across Acc & Gyro Axes")
    plt.grid(True)

    plt.axhline(threshold, color='red', linestyle='--', label=f'Threshold = {threshold}')

    plt.legend()
    plt.tight_layout()
    plt.show()


plot_imu_variance(X_train, X_test, sample_idx=sample_index, source='train', window=10, threshold=0.15)



#segment to retain active region
from scipy.signal import resample

def segment_writing_phases(
    x_train,
    x_test,
    window_size=5,
    threshold=0.5
):

    def find_active_segment(sample, window_size, threshold):
        # Convert the sample to a DataFrame for rolling variance
        df = pd.DataFrame(
            sample,
            columns=["Accx", "Accy", "Accz",'GyroX','GyroY','GyroZ']
        )

        # Rolling variance across time, then mean over 6 axes
        rolling_var = df.rolling(window=window_size, center=True).var().mean(axis=1)

        # Boolean mask where variance > threshold
        active_mask = rolling_var > threshold
        active_indices = np.where(active_mask)[0]

        if len(active_indices) == 0:
            # No active region
            return None, None
        else:
            return active_indices[0], active_indices[-1]

    def truncate_samples(x_array):
        truncated_list = []
        for sample in x_array:
            start_idx, end_idx = find_active_segment(sample, window_size, threshold)
            if start_idx is not None:
                # Truncate to [start_idx : end_idx + 1]
                truncated_sample = sample[start_idx:end_idx+1]
            else:
                # No active region found => return the whole sample
                truncated_sample = sample
            truncated_list.append(truncated_sample)
        return truncated_list

    # 1) Truncate train and test
    truncated_train_list = truncate_samples(x_train)
    truncated_test_list = truncate_samples(x_test)

    # 2) Compute median length *from the training set only*
    train_lengths = [len(s) for s in truncated_train_list]
    median_len_train = int(np.median(train_lengths)) if len(train_lengths) > 0 else 0

    def resample_to_length(sample_list, desired_length):
        resampled_list = []
        for samp in sample_list:
            if desired_length > 0:
                # Resample along time dimension (axis=0)
                samp_resampled = resample(samp, desired_length, axis=0)
            else:
                # If desired_length = 0, produce an empty shape (0, 6)
                samp_resampled = np.zeros((0, samp.shape[1]), dtype=samp.dtype)
            resampled_list.append(samp_resampled)
        return resampled_list
    print("median len: ",median_len_train)

    # 3) Resample both train and test to the same median length from train
    resampled_train_list = resample_to_length(truncated_train_list, 46) #supposed to be median_len_train
    resampled_test_list  = resample_to_length(truncated_test_list,  46)

    # 4) Stack the list of resampled arrays into a single np.ndarray
    #    Now all samples have the same time dimension
    x_train_out = np.stack(resampled_train_list, axis=0)
    x_test_out  = np.stack(resampled_test_list,  axis=0)

    return x_train_out, x_test_out

X_train_truncated, X_test_truncated = segment_writing_phases(
    X_train,
    X_test,
    window_size=10,
    threshold=0.15
)


# picking up sample to show
sample_data_truncated = X_train_truncated[sample_index]  # Shape: (time_steps, features)

# Showing nature of data after normalising
plt.figure(figsize=(12, 6))
for feature_idx in range(sample_data_truncated.shape[1]):
    plt.plot(sample_data_truncated[:, feature_idx], label=f"{features[feature_idx]}")
plt.title(f"Time-Series Data for Sample {sample_index} (truncated)")
plt.xlabel("Time Steps")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()

In [ ]:
# merging main dataset wtih nac
from sklearn.utils import shuffle

# Merge the X_train datasets along the first axis (samples)
X_train = np.concatenate((X_train, nac_X_train), axis=0)
X_test = np.concatenate((X_test, nac_X_test), axis=0)

# Merge the y_train datasets
y_train = np.concatenate((y_train, nac_y_train), axis=0)
y_test = np.concatenate((y_test, nac_y_test), axis=0)

# Shuffle the merged datasets while maintaining correspondence between X and y
X_train, y_train = shuffle(X_train, y_train, random_state=42)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Checking dimensions of the input after preprocessing
print(f"Processed X_train shape: {len(X_train)} samples, first sample shape: {X_train[0].shape}")
print(f"Processed X_test shape: {len(X_test)} samples, first sample shape: {X_test[0].shape}")

# Encoding labels using LabelEncoder so there is no hierarchy of weights for each label
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Ensuring input shape is appropriate
time_steps = X_train.shape[1]
features = X_train.shape[2]
num_classes = 27 #since there are 26 alphabets and 1 nac(not a character)


early_stopping_cnn = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
early_stopping_lstm = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
early_stopping_bilstm = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)


#for each of the models below, our decision on number of layers, neurons, regularisation, etc were based on research and our own
#experimentation and tweaking parameters for optimal results. 

# CNN Model
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=32, activation='relu'),
        MaxPooling1D(pool_size=16), 
        Dropout(0.6),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    return model


# LSTM Model
def build_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128,unroll=True, return_sequences=False),
        Dropout(0.6),
        Dense(num_classes, activation='softmax')
    ])
    return model

# BiLSTM Model
def build_bilstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(128)),
        Dropout(0.6),
        Dense(num_classes, activation='softmax')
    ])
    return model


# Compiling all models with the Adam optimiser
cnn_model = build_cnn_model((time_steps, features), num_classes)
cnn_model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lstm_model = build_lstm_model((time_steps, features), num_classes)
lstm_model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

bilstm_model = build_bilstm_model((time_steps, features), num_classes)
bilstm_model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Training and evaluating the CNN model
print("Training CNN model...")
cnn_history = cnn_model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=5000, batch_size=32, callbacks=[early_stopping_cnn])
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(X_test, y_test_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy * 100:.2f}%")

# Training and evaluating the LSTM model
print("Training LSTM model...")
lstm_history = lstm_model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=5000, batch_size=32, callbacks=[early_stopping_lstm])
lstm_test_loss, lstm_test_accuracy = lstm_model.evaluate(X_test, y_test_encoded)
print(f"LSTM Test Accuracy: {lstm_test_accuracy * 100:.2f}%")

# Train and evaluating the BiLSTM model
print("Training BiLSTM model...")
bilstm_history = bilstm_model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=5000, batch_size=32, callbacks=[early_stopping_bilstm])
bilstm_test_loss, bilstm_test_accuracy = bilstm_model.evaluate(X_test, y_test_encoded)
print(f"BiLSTM Test Accuracy: {bilstm_test_accuracy * 100:.2f}%")


In [ ]:
#confusion matrices for each model
class_names = label_encoder.classes_
# Predicting and generating confusion matrix for CNN
cnn_y_pred = np.argmax(cnn_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, cnn_y_pred, "CNN", class_names)

# Predicting and generating confusion matrix for LSTM
lstm_y_pred = np.argmax(lstm_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, lstm_y_pred, "LSTM", class_names)

# Predicting and generating confusion matrix for BiLSTM
bilstm_y_pred = np.argmax(bilstm_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, bilstm_y_pred, "BiLSTM", class_names)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, model_name, class_names):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalize to percentages (0-100)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Increase figure size and adjust annotations
    plt.figure(figsize=(14, 12)) 
    
    sns.heatmap(
        cm_percent, 
        annot=True, 
        fmt='.1f',    
        annot_kws={'size': 12}, 
        cmap='Blues',
        xticklabels=class_names, 
        yticklabels=class_names,
        cbar=False,
        square=True        
    )
    
    # Adjust label and title font sizes
    plt.title(f'Confusion Matrix for {model_name} (%)', fontsize=16)
    plt.ylabel('True Label', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=14)
    
    # Rotate tick labels for better readability
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(rotation=0, fontsize=12)
    
    # Add padding to prevent cutoff
    plt.tight_layout()
    plt.show()


class_names = label_encoder.classes_

# CNN
cnn_y_pred = np.argmax(cnn_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, cnn_y_pred, "CNN", class_names)

# LSTM
lstm_y_pred = np.argmax(lstm_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, lstm_y_pred, "LSTM", class_names)

# BiLSTM
bilstm_y_pred = np.argmax(bilstm_model.predict(X_test), axis=1)
plot_confusion_matrix(y_test_encoded, bilstm_y_pred, "BiLSTM", class_names)

In [ ]:
print(f"CNN Test Accuracy: {cnn_test_accuracy * 100:.2f}%")

print(f"LSTM Test Accuracy: {lstm_test_accuracy * 100:.2f}%")

print(f"BiLSTM Test Accuracy: {bilstm_test_accuracy * 100:.2f}%")

In [ ]:
def plot_training_history(history, model_name):
    # Extract data from history
    acc = history.history.get('accuracy')
    val_acc = history.history.get('val_accuracy')
    loss = history.history.get('loss')
    val_loss = history.history.get('val_loss')
    epochs = range(1, len(acc) + 1)

    # Plotting Accuracy
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title(model_name+' Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Plotting Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title( model_name+' Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Display the plots
    plt.tight_layout()
    plt.show()

plot_training_history(cnn_history, "CNN")

plot_training_history(lstm_history, "LSTM")

plot_training_history(bilstm_history, "BiLSTM")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

#Print macro-averaged, micro-averaged, or weighted metrics
macro_precision = precision_score(y_test_encoded, cnn_y_pred, average='macro')
macro_recall    = recall_score(y_test_encoded, cnn_y_pred, average='macro')
macro_f1        = f1_score(y_test_encoded, cnn_y_pred, average='macro')

print("Macro Precision:", macro_precision)
print("Macro Recall:   ", macro_recall)
print("Macro F1-score: ", macro_f1)

# Print a classification report (precision, recall, F1 for each class + overall)
print("\nDetailed classification report :")
print(classification_report(y_test_encoded, cnn_y_pred, target_names=class_names))

################################################################################################
# Print macro-averaged, micro-averaged, or weighted metrics
macro_precision = precision_score(y_test_encoded, lstm_y_pred, average='macro')
macro_recall    = recall_score(y_test_encoded, lstm_y_pred, average='macro')
macro_f1        = f1_score(y_test_encoded, lstm_y_pred, average='macro')

print("Macro Precision:", macro_precision)
print("Macro Recall:   ", macro_recall)
print("Macro F1-score: ", macro_f1)

# Print a classification report (precision, recall, F1 for each class + overall)
print("\nDetailed classification report LSTM:")
print(classification_report(y_test_encoded, lstm_y_pred, target_names=class_names))


#############################################################################################
# Print macro-averaged, micro-averaged, or weighted metrics
macro_precision = precision_score(y_test_encoded, bilstm_y_pred, average='macro')
macro_recall    = recall_score(y_test_encoded, bilstm_y_pred, average='macro')
macro_f1        = f1_score(y_test_encoded, bilstm_y_pred, average='macro')

print("Macro Precision:", macro_precision)
print("Macro Recall:   ", macro_recall)
print("Macro F1-score: ", macro_f1)

# Print a classification report (precision, recall, F1 for each class + overall)
print("\nDetailed classification report BiLSTM:")
print(classification_report(y_test_encoded, bilstm_y_pred, target_names=class_names))